In [1]:
import numpy as n

In [2]:
#获取数据
def get_data(filename):
    lines = open(filename,'r').readlines()
    items = []
    for l in lines:
        tmp = l.strip() #删除字符串头尾空白字符
        item = [i for i in tmp.split(",")] #分割元素转成数组 >>[0, 4828, 774, 207, 7460, 7465, 3768, 10221, 22435]
        items.append(item)   

    return items

Demand = get_data("../data/demand.csv")
Qos = get_data("../data/qos.csv")
Site_bandwidth = get_data("../data/site_bandwidth.csv")

In [3]:
class Site:
    def __init__(self):
        self.bandwidth = 0
        self.overload = False #1超载
        self.free = 0 #剩余带宽


In [4]:
#用户需求
demand_h = Demand[0][1:]
demand_v = [i[1:] for i in Demand[1:]]
#初始化节点
Sites = dict()
for s in Site_bandwidth[1:]:
    site = Site()
    site.bandwidth = int(s[1])
    site.free = int(s[1])
    Sites[s[0]] = site

sites = Sites


In [10]:
# #获得user候选site
# user_candidate = dict()
# qos_h = Qos[0][1:]
#
# for u in qos_h:
#     user_candidate[u] = dict()
#
# for item in Qos[1:]:
#     s = item[0]
#     for index,j in enumerate(item[1:]):
#         if int(j) < 400:
#             user_candidate[qos_h[index]][s] = int(j)
#
# #计算每个user候选site数
# user_Nsite = dict()
# for i in user_candidate:
#     #print(i, len(user_candidate[i]))
#     #候选site数
#     user_Nsite[i] = len(user_candidate[i])
#     #free bandwidth 升序
#     user_candidate[i] = sorted(user_candidate[i].items(), key=lambda d: d[1], reverse=False)
# #site数升序
# user_Nsite = sorted(user_Nsite.items(), key=lambda d: d[1], reverse=False)

In [5]:
class User:
    def __init__(self):
        self.candidate_sites = dict()
        self.n = 0

users = dict()
qos_h = Qos[0][1:]

for u in qos_h:
    users[u] = User()

for item in Qos[1:]:
    s = item[0] #节点名
    for index,u in enumerate(item[1:]):
        if int(u) < 400: #筛选符合user条件的节点
            users[qos_h[index]].n += 1
            users[qos_h[index]].candidate_sites[s] = sites[s]
#user按site数排序
users = sorted(users.items(),key=lambda l:l[1].n, reverse=False)
users = {u[0]:u[1] for u in users}
#候选site按free bandwidth排序
def update_freesite(users,sites):
    for u in users:
        tmp = users[u].candidate_sites
        for s in tmp:
            tmp[s].free = sites[s].free #更新candidate表里的数据
        tmp = sorted(tmp.items(),key=lambda l:l[1].free, reverse=False)
        users[u].candidate_sites = {u[0]:u[1] for u in tmp}
    return users

users = update_freesite(users,sites)

In [9]:
for demand in demand_v:
    print("---")
    users_tmp = users
    sites = Sites
    for u in users_tmp:
        user_demand = int(demand[demand_h.index(u)])
        if user_demand == 0:
            print(u)
            continue
        out = dict()
        for s in users_tmp[u].candidate_sites:

            if sites[s].overload:
                continue
            if sites[s].free > user_demand: #空闲带宽多则直接分配,并服务下个用户
                out[s] = user_demand
                sites[s].free = sites[s].free - user_demand
                break
            else: #否则接着找下个节点
                out[s] = sites[s].free
                user_demand = user_demand - sites[s].free
                sites[s].overload = 1
        print("%s:"%u,end="")
        for k,v in out.items():
            print("<{},{}>".format(k,v),end=" ")
        print('')
        #服务完一个user后更新sites空余带宽并升序
        users_tmp = update_freesite(users_tmp,sites)

---
B:<AD,17223> 
E:<Dr,6070> 
F:<Dp,34940> 
H:<AS,3180> 
C:<AS,8581> 
D:<AS,11671> 
G:<AS,9969> 
A:<AS,19261> 
J:<AS,39796> 
I:<AS,32248> 
---
B:<AD,16160> 
E:<Dr,8601> 
F:<Dp,49344> 
H:<AS,2092> 
C:<AS,6409> 
D:<AS,11106> 
G:<AS,6793> <AV,3982> 
A:<AV,11964> 
J:<AV,41054> 
I:<Dp,22096> 
---
B:<AD,24221> 
E:<Dr,5568> 
F:<Dp,34047> 
H:<AV,2992> 
C:<AV,6250> 
D:<AV,10323> 
G:<AV,10646> 
A:<AV,15121> 
J:<AV,37494> 
I:<Dp,23312> 
---
B:<AD,15743> 
E:<Dr,7515> 
F:<Dp,63096> 
H:<AV,3427> 
C:<AV,5777> 
D:<AV,11501> 
G:<AV,9761> 
A:<AV,12804> 
J:<AV,31812> 
I:<Dp,32385> 
---
B:<AD,17188> 
E:<Dr,4872> 
F:<Dp,32024> 
H:<AV,3123> 
C:<AV,5831> 
D:<AV,9453> 
G:<AV,7260> 
A:<AV,11204> 
J:<AV,28230> 
I:<Dp,26458> 
---
B:<AD,15832> 
E:<Dr,6115> 
F:<Dp,56767> <BM,10890> 
H:<AV,2533> 
C:<AV,5120> 
D:<AV,6419> 
G:<AV,6275> 
A:<AV,12083> 
J:<AV,33670> 
I:<AV,26195> 
---
B:<AD,18072> 
E:<Dr,5207> 
F:<BM,34671> 
H:<AV,3058> 
C:<AV,7362> 
D:<AV,5077> 
G:<AV,9736> 
A:<AV,15315> 
J:<AV,17228> <BM,11616> 
I:<B

In [51]:
print(out)

{'Dp': 17196}
